In [1]:
def fizz_buzz_encode(i):
    if i % 15 == 0:
        return 3
    elif i % 5 == 0:
        return 2
    elif i % 3 == 0:
        return 1
    else:
        return 0


def fizz_buzz_decode(i, prediction):
    return [str(i), 'fizz', 'buzz', 'fizzbuzz'][prediction]

In [2]:
print(fizz_buzz_decode(1, fizz_buzz_encode(1)))
print(fizz_buzz_decode(2, fizz_buzz_encode(2)))
print(fizz_buzz_decode(5, fizz_buzz_encode(5)))
print(fizz_buzz_decode(12, fizz_buzz_encode(12)))
print(fizz_buzz_decode(15, fizz_buzz_encode(15)))

1
2
buzz
fizz
fizzbuzz


In [3]:
def helper(i):
    print(fizz_buzz_decode(i,fizz_buzz_encode(i)))

for i in range(1, 16):
    helper(i)

1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz


In [4]:
import numpy as np
import torch

In [5]:
NUM_DIGITS=10

def binary_encode(i,num_digits):
    return np.array([i>>d & 1 for d in range(num_digits)][::-1])

trX=torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2**NUM_DIGITS)])
trY=torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2**NUM_DIGITS)])
print(binary_encode(31, NUM_DIGITS))

[0 0 0 0 0 1 1 1 1 1]


In [6]:
NUM_HIDDEN=100
model=torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)
if torch.cuda.is_available():
    model=model.cuda()

loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
BATCH_SIZE=128
for epoch in range(10000):
    for start in range(0, len(trX), BATCH_SIZE):
        end=start+BATCH_SIZE
        batchX=trX[start:end]
        batchY=trY[start:end]

        if torch.cuda.is_available():
            batchX=batchX.cuda()
            batchY=batchY.cuda()

        y_pred=model(batchX)
        loss=loss_fn(y_pred,batchY)

        print('Epoch', epoch, loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

testX=torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
if torch.cuda.is_available():
    testX=testX.cuda()
with torch.no_grad():
    testY=model(testX)

predictions=zip(range(0,101), testY.max(1)[1].cpu().data.numpy())
print([fizz_buzz_decode(i,x) for i , x in predictions])

Epoch 0 1.4727123975753784
Epoch 0 1.4698920249938965
Epoch 0 1.4507133960723877
Epoch 0 1.4421011209487915
Epoch 0 1.4194750785827637
Epoch 0 1.4085015058517456
Epoch 0 1.3810805082321167
Epoch 0 1.3752740621566772
Epoch 1 1.3496999740600586
Epoch 1 1.3427318334579468
Epoch 1 1.314843773841858
Epoch 1 1.3250226974487305
Epoch 1 1.298940658569336
Epoch 1 1.2851145267486572
Epoch 1 1.2607252597808838
Epoch 1 1.2313010692596436
Epoch 2 1.2581870555877686
Epoch 2 1.2528140544891357
Epoch 2 1.221522331237793
Epoch 2 1.243546485900879
Epoch 2 1.2153505086898804
Epoch 2 1.20371675491333
Epoch 2 1.1889113187789917
Epoch 2 1.1508753299713135
Epoch 3 1.1963976621627808
Epoch 3 1.1968265771865845
Epoch 3 1.1675487756729126
Epoch 3 1.1957285404205322
Epoch 3 1.1677541732788086
Epoch 3 1.160892367362976
Epoch 3 1.1586273908615112
Epoch 3 1.1234400272369385
Epoch 4 1.161698579788208
Epoch 4 1.168649435043335
Epoch 4 1.1437699794769287
Epoch 4 1.174160361289978
Epoch 4 1.1482535600662231
Epoch 4 1.1

Epoch 49 1.0969467163085938
Epoch 49 1.109716534614563
Epoch 49 1.1052850484848022
Epoch 49 1.1040363311767578
Epoch 49 1.1279176473617554
Epoch 49 1.0345004796981812
Epoch 50 1.0939024686813354
Epoch 50 1.1036288738250732
Epoch 50 1.0949515104293823
Epoch 50 1.1080322265625
Epoch 50 1.103613257408142
Epoch 50 1.1026298999786377
Epoch 50 1.1264578104019165
Epoch 50 1.0316442251205444
Epoch 51 1.092236876487732
Epoch 51 1.1019237041473389
Epoch 51 1.093263030052185
Epoch 51 1.1057255268096924
Epoch 51 1.1020475625991821
Epoch 51 1.1012489795684814
Epoch 51 1.125233769416809
Epoch 51 1.0305968523025513
Epoch 52 1.0905516147613525
Epoch 52 1.0997878313064575
Epoch 52 1.0915926694869995
Epoch 52 1.1038841009140015
Epoch 52 1.1002618074417114
Epoch 52 1.099584937095642
Epoch 52 1.1238478422164917
Epoch 52 1.0278403759002686
Epoch 53 1.0889711380004883
Epoch 53 1.097652792930603
Epoch 53 1.0900906324386597
Epoch 53 1.1019749641418457
Epoch 53 1.0989896059036255
Epoch 53 1.09835684299469
Epoc

Epoch 100 0.9939588308334351
Epoch 100 0.9822402000427246
Epoch 100 0.9979297518730164
Epoch 100 0.9917858839035034
Epoch 100 1.0461616516113281
Epoch 100 0.8969132304191589
Epoch 101 0.979953408241272
Epoch 101 0.9682893753051758
Epoch 101 0.990990936756134
Epoch 101 0.9784642457962036
Epoch 101 0.9958038330078125
Epoch 101 0.9885669946670532
Epoch 101 1.0430774688720703
Epoch 101 0.8946582674980164
Epoch 102 0.9770093560218811
Epoch 102 0.9645233154296875
Epoch 102 0.9881782531738281
Epoch 102 0.9755128026008606
Epoch 102 0.9922965168952942
Epoch 102 0.985410213470459
Epoch 102 1.042035698890686
Epoch 102 0.8916453719139099
Epoch 103 0.9732375144958496
Epoch 103 0.9606217741966248
Epoch 103 0.984960675239563
Epoch 103 0.9728078842163086
Epoch 103 0.9905120730400085
Epoch 103 0.9826086163520813
Epoch 103 1.0385292768478394
Epoch 103 0.8879762291908264
Epoch 104 0.9702709913253784
Epoch 104 0.9573346376419067
Epoch 104 0.9823812246322632
Epoch 104 0.9690788984298706
Epoch 104 0.9869186

Epoch 152 0.8508254885673523
Epoch 152 0.8219704031944275
Epoch 152 0.8551363348960876
Epoch 152 0.8432700037956238
Epoch 152 0.9272533059120178
Epoch 152 0.7293758988380432
Epoch 153 0.825628936290741
Epoch 153 0.7977834939956665
Epoch 153 0.84714674949646
Epoch 153 0.8188021779060364
Epoch 153 0.852604329586029
Epoch 153 0.840614914894104
Epoch 153 0.9248632192611694
Epoch 153 0.7260476350784302
Epoch 154 0.8229225873947144
Epoch 154 0.7949913144111633
Epoch 154 0.8445956110954285
Epoch 154 0.8159909248352051
Epoch 154 0.8493919372558594
Epoch 154 0.8370196223258972
Epoch 154 0.9221429228782654
Epoch 154 0.7222760319709778
Epoch 155 0.8200381398200989
Epoch 155 0.792194128036499
Epoch 155 0.8419153094291687
Epoch 155 0.812062680721283
Epoch 155 0.8463185429573059
Epoch 155 0.8348990678787231
Epoch 155 0.9205503463745117
Epoch 155 0.7199294567108154
Epoch 156 0.8176100850105286
Epoch 156 0.7891151905059814
Epoch 156 0.8391171097755432
Epoch 156 0.8106213212013245
Epoch 156 0.844260156

Epoch 202 0.7187604904174805
Epoch 202 0.7156831622123718
Epoch 202 0.8123117685317993
Epoch 202 0.579467236995697
Epoch 203 0.6961971521377563
Epoch 203 0.6635921001434326
Epoch 203 0.7187255620956421
Epoch 203 0.6765331625938416
Epoch 203 0.7173627018928528
Epoch 203 0.7143056988716125
Epoch 203 0.8099403977394104
Epoch 203 0.5756126642227173
Epoch 204 0.6937122941017151
Epoch 204 0.6615262627601624
Epoch 204 0.7170805931091309
Epoch 204 0.6758158802986145
Epoch 204 0.7134652733802795
Epoch 204 0.7111244797706604
Epoch 204 0.8081948757171631
Epoch 204 0.5729444026947021
Epoch 205 0.6920348405838013
Epoch 205 0.6580454707145691
Epoch 205 0.7141613960266113
Epoch 205 0.672536313533783
Epoch 205 0.7120054960250854
Epoch 205 0.7102451324462891
Epoch 205 0.8067906498908997
Epoch 205 0.5702157020568848
Epoch 206 0.6891707181930542
Epoch 206 0.6560742855072021
Epoch 206 0.7129535675048828
Epoch 206 0.6694170832633972
Epoch 206 0.7083630561828613
Epoch 206 0.7069778442382812
Epoch 206 0.8043

...

Epoch 9779 8.633334687146998e-07
Epoch 9779 1.942668887977561e-07
Epoch 9780 2.589075620562653e-07
Epoch 9780 4.7311110051850847e-07
Epoch 9780 5.988397902001452e-07
Epoch 9780 4.5262211756380566e-07
Epoch 9780 3.5576482559918077e-07
Epoch 9780 3.7811639685969567e-07
Epoch 9780 8.642647912893153e-07
Epoch 9780 1.8985173255714471e-07
Epoch 9781 2.5611359433241887e-07
Epoch 9781 4.7124848379098694e-07
Epoch 9781 5.969770882074954e-07
Epoch 9781 4.5075952925799356e-07
Epoch 9781 3.557648540208902e-07
Epoch 9781 3.7904771943431115e-07
Epoch 9781 8.670587590131618e-07
Epoch 9781 1.942668887977561e-07
Epoch 9782 2.5611356591070944e-07
Epoch 9782 4.6845454448884993e-07
Epoch 9782 5.941831773270678e-07
Epoch 9782 4.5169085183260904e-07
Epoch 9782 3.566961765955057e-07
Epoch 9782 3.7904771943431115e-07
Epoch 9782 8.661274932819651e-07
Epoch 9782 1.942668887977561e-07
Epoch 9783 2.542509491831879e-07
Epoch 9783 4.7217983478731185e-07
Epoch 9783 5.941832341704867e-07
Epoch 9783 4.50759557679703e-

Epoch 9826 4.1350673996021214e-07
Epoch 9826 3.1013013312986004e-07
Epoch 9826 3.4552022043499164e-07
Epoch 9826 7.869655291870004e-07
Epoch 9826 1.8543659052738803e-07
Epoch 9827 2.346932035379723e-07
Epoch 9827 4.330644287620089e-07
Epoch 9827 5.438918719846697e-07
Epoch 9827 4.144380625348276e-07
Epoch 9827 3.11992778279091e-07
Epoch 9827 3.436575752857607e-07
Epoch 9827 7.878968517616158e-07
Epoch 9827 1.8985174676799943e-07
Epoch 9828 2.346932035379723e-07
Epoch 9828 4.3306437191859004e-07
Epoch 9828 5.448231945592852e-07
Epoch 9828 4.144380625348276e-07
Epoch 9828 3.11992778279091e-07
Epoch 9828 3.4645148616618826e-07
Epoch 9828 7.823089731573418e-07
Epoch 9828 1.8543659052738803e-07
Epoch 9829 2.337618667525021e-07
Epoch 9829 4.284078158889315e-07
Epoch 9829 5.420292268354387e-07
Epoch 9829 4.1350673996021214e-07
Epoch 9829 3.138554518500314e-07
Epoch 9829 3.436575752857607e-07
Epoch 9829 7.878968517616158e-07
Epoch 9829 1.8543657631653332e-07
Epoch 9830 2.3376188096335682e-07
E

Epoch 9877 3.902236755948252e-07
Epoch 9877 4.889439537691942e-07
Epoch 9877 3.8277318026302964e-07
Epoch 9877 2.7846522243635263e-07
Epoch 9877 3.082674311372102e-07
Epoch 9877 7.022155728009238e-07
Epoch 9877 1.5453049684310827e-07
Epoch 9878 2.132727843218163e-07
Epoch 9878 3.8556711956516665e-07
Epoch 9878 4.898752195003908e-07
Epoch 9878 3.781166242333711e-07
Epoch 9878 2.719459644140443e-07
Epoch 9878 3.091987537118257e-07
Epoch 9878 7.022155728009238e-07
Epoch 9878 1.5894565308371966e-07
Epoch 9879 2.123414617472008e-07
Epoch 9879 3.892923530202097e-07
Epoch 9879 4.861499860453478e-07
Epoch 9879 3.753226565095247e-07
Epoch 9879 2.7473996055960015e-07
Epoch 9879 3.1106139886105666e-07
Epoch 9879 6.96627637353231e-07
Epoch 9879 1.6777596556494245e-07
Epoch 9880 2.0954749402335437e-07
Epoch 9880 3.902236755948252e-07
Epoch 9880 4.852186634707323e-07
Epoch 9880 3.753225996661058e-07
Epoch 9880 2.728773154103692e-07
Epoch 9880 3.0733610856259475e-07
Epoch 9880 6.966276941966498e-07
E

Epoch 9927 2.7567119786908734e-07
Epoch 9927 6.258474627429678e-07
Epoch 9927 1.3686987188066269e-07
Epoch 9928 1.8998974837813876e-07
Epoch 9928 3.4272630955456407e-07
Epoch 9928 4.358586522812402e-07
Epoch 9928 3.3341322591695643e-07
Epoch 9928 2.4214369886976783e-07
Epoch 9928 2.728772301452409e-07
Epoch 9928 6.211909067133092e-07
Epoch 9928 1.4570017015103076e-07
Epoch 9929 1.8440181293044589e-07
Epoch 9929 3.455202772784105e-07
Epoch 9929 4.358586238595308e-07
Epoch 9929 3.315505807677255e-07
Epoch 9929 2.412123478734429e-07
Epoch 9929 2.775338714400277e-07
Epoch 9929 6.221221724445058e-07
Epoch 9929 1.4570017015103076e-07
Epoch 9930 1.8719578065429232e-07
Epoch 9930 3.4552033412182936e-07
Epoch 9930 4.330646845573938e-07
Epoch 9930 3.3620716521909344e-07
Epoch 9930 2.402810537205369e-07
Epoch 9930 2.7194593599233485e-07
Epoch 9930 6.202595841386938e-07
Epoch 9930 1.3686987188066269e-07
Epoch 9931 1.834704903558304e-07
Epoch 9931 3.4272630955456407e-07
Epoch 9931 4.339960071320092

Epoch 9977 2.495942226232728e-07
Epoch 9977 5.494793526850117e-07
Epoch 9977 1.236244031588285e-07
Epoch 9978 1.704319743112137e-07
Epoch 9978 3.064048144096887e-07
Epoch 9978 3.8556729009542323e-07
Epoch 9978 2.9895431907789316e-07
Epoch 9978 2.0954746560164494e-07
Epoch 9978 2.449376097501954e-07
Epoch 9978 5.466853849611653e-07
Epoch 9978 1.192092469182171e-07
Epoch 9979 1.6950065173659823e-07
Epoch 9979 3.0174822995832074e-07
Epoch 9979 3.8929258039388515e-07
Epoch 9979 2.9057241590635385e-07
Epoch 9979 2.0954745139079023e-07
Epoch 9979 2.430749930226739e-07
Epoch 9979 5.438914172373188e-07
Epoch 9979 1.236244031588285e-07
Epoch 9980 1.6577536143813631e-07
Epoch 9980 2.998855848090898e-07
Epoch 9980 3.8463596752080775e-07
Epoch 9980 2.933663836302003e-07
Epoch 9980 2.067534836669438e-07
Epoch 9980 2.3934970272421197e-07
Epoch 9980 5.38303481789626e-07
Epoch 9980 1.192092469182171e-07
Epoch 9981 1.6577536143813631e-07
Epoch 9981 3.0454216926045774e-07
Epoch 9981 3.874299352446542e-0